In [ ]:
from dotenv import load_dotenv
import os #provides ways to access the Operating System and allows us to read the environment variables

load_dotenv()

GOOGLE_AI_KEY = os.getenv("GOOGLE_AI_KEY_021124")

##**Install Langchain and Google Gemini related libraries**

In [3]:
!pip install -q -U langchain
!pip install -q -U google-generativeai langchain-google-genai

Reshimming asdf python...
Reshimming asdf python...


##**Get Gemini Key from Secrets **
Set GEMINI_KEY secret key at Google Colab and get that here to runn Google Gemini LLM. You can get Google Gemini Key from following link https://makersuite.google.com/app/apikey

In [ ]:
# from google.colab import userdata
GOOGLE_API_KEY = GOOGLE_AI_KEY

##**Run Simple LLM Prompt**

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, convert_system_message_to_human=True)

chain = prompt | llm

In [5]:
resp = chain.invoke({"input": "how can langsmith help with testing?"})
print(resp)

content=" As a world-class technical documentation writer, Langsmith can assist with testing in the following ways:\n\n1. **Test Planning:**\n   - Collaborate with the testing team to understand the testing strategy and objectives.\n   - Review testing plans and provide feedback to ensure they align with the documentation goals.\n\n2. **Test Case Creation:**\n   - Develop clear and concise test cases based on the requirements and specifications.\n   - Ensure that test cases cover various scenarios and edge cases.\n   - Write test cases in a structured format that is easy to understand and execute.\n\n3. **Test Procedure Documentation:**\n   - Create detailed test procedures that provide step-by-step instructions for executing test cases.\n   - Include screenshots, diagrams, and other visuals to aid in understanding the test steps.\n   - Ensure that test procedures are comprehensive and unambiguous to avoid any confusion during testing.\n\n4. **Test Result Reporting:**\n   - Develop tem

##**Run LLM with OutputParser**

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [7]:
chain = prompt | llm | output_parser

In [8]:
resp = chain.invoke({"input": "how can langsmith help with testing?"})
print(resp)

LangSmith can help with testing in a variety of ways:

1. **Automated Testing:** LangSmith can be used to generate automated tests for your software applications. This can save time and effort compared to manually writing and executing tests.

2. **Cross-platform Testing:** LangSmith can be used to generate tests that run on multiple platforms, including Windows, Linux, and macOS. This can help to ensure that your software works correctly on all of the platforms that it is intended to support.

3. **Performance Testing:** LangSmith can be used to generate tests that measure the performance of your software applications. This can help to identify areas where performance can be improved.

4. **Security Testing:** LangSmith can be used to generate tests that check for security vulnerabilities in your software applications. This can help to identify and fix vulnerabilities before they can be exploited by attackers.

5. **Localization Testing:** LangSmith can be used to generate tests that 

##**Run Retrieval Augumented Generation with LangChain and Gemini**
We are using beautifulsoup4 to scrap web data and use that as external source to augment to LLM

In [ ]:
!pip install beautifulsoup4

In [9]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [10]:
# !pip install -q -U sentence-transformers
from langchain.embeddings import HuggingFaceEmbeddings
# Using HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

In [ ]:
!pip install faiss-gpu

In [11]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [13]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by letting you visualize test results.'

In [14]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [16]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith assists with testing in the following ways:

1. Dataset uploading: LangSmith simplifies dataset uploading, making it easy to use a constructed dataset or create a small dataset manually.

2. Visualizing outputs: The most basic approach to testing changes is to run the chain over the data points and visualize the outputs. LangSmith facilitates this process by allowing users to pull down a dataset and run a chain over them, logging the results to a new project.

3. Assigning feedback and marking runs: LangSmith allows users to assign feedback to runs and mark them as correct or incorrect directly in the web app, displaying aggregate statistics for each test project.

4. Using evaluators: LangSmith includes a set of evaluators in the open-source LangChain library that can be specified when initiating a test run. These evaluators evaluate the results once the test run is complete, guiding users to examples that they should look at.

5. Manual review and annotation: LangSmith enab

##**Run Converstation Retrieval chain with history**
The provided way in LangChain quickstart guide throws error when we use Google Gemini with LangChain. There might be support issue of Langchain with Google Gemini. This is alternative method.

In [18]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=100)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2'))

In [19]:
from langchain.chains import ConversationalRetrievalChain

# Connect query to FAISS index using a retriever
retriever = vector.as_retriever()

qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever,return_source_documents=True)
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7143a1a33cd0>


In [20]:
import sys
chat_history = []
while True:
    query = input('Prompt: ')
    result = qa_chain({'question': query, 'chat_history': chat_history})
    print('Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))

/home/daryl/.asdf/installs/python/3.11.3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Answer: I cannot find the answer to your question in the provided context.

Answer: I do not have enough information to answer this question. The context does not contain any information to rephrase a standalone question.

Answer: I do not have enough information to answer this question. The context does not mention anything about rephrasing a standalone question.

Answer: I cannot find the answer to your question in the context provided.

Answer: I'm sorry, but I cannot find the answer to your question within the provided context. The context mainly describes how to use LangSmith to debug, monitor, and evaluate LLM applications. It does not provide information about the impact of changing the prompt on the output of an LLM application.

Answer: I cannot find the answer to your question in the provided context.

Answer: I cannot answer this question as the provided context does not contain the answer.

Answer: I do not have enough information to answer this question.

Answer: I do not 